In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
# DELETE FILES FROM WORKING DIRECTORY

# import zipfile
# import os

# # Define the path where the zip file is stored
# zip_path = '/kaggle/input/your-dataset-name/your-file.zip'
# extract_path = '/kaggle/working/your-extracted-folder/'

# # Unzip the file
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# # List the files extracted to ensure it worked
# os.listdir(extract_path)

# import os

# # Specify the path of the file you want to delete
# # file_path = '/kaggle/input/train-data/train.py'


# dst_dir = '/kaggle/working/'  # Destination directory
# # os.listdir(dst_dir)


# # Iterate over all the files in the source directory
# for filename in os.listdir(dst_dir):
#     dst_file = os.path.join(dst_dir, filename)
    
#     # Ensure that it's a file (not a directory)
#     if os.path.isfile(dst_file):
#         os.remove(dst_file)

import os
import shutil

dst_dir = '/kaggle/working/'  # Destination directory

# Iterate over all the files and directories in the destination directory
for filename in os.listdir(dst_dir):
    dst_file = os.path.join(dst_dir, filename)
    
    # Skip hidden directories (those starting with a dot)
    if filename.startswith('.'):
        continue
    
    # If it's a file, remove it
    if os.path.isfile(dst_file):
        os.remove(dst_file)
    # If it's a directory, remove it
    elif os.path.isdir(dst_file):
        shutil.rmtree(dst_file)  # This will remove the directory and all its contents


In [ ]:
# !ls /kaggle/input/
!pwd

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
# from models import TwoStreamNetworkTransferLearning
from models_new import TwoStreamNetworkTransferLearning
from data_loader_new_train import get_data_loaders
import json
import os
import time

def train_model(num_epochs, frame_direc, device, batch_size=1, model_name=None):
    print("Entering to train data!")
    model = TwoStreamNetworkTransferLearning().to(device)
    
    # Load pre-trained model if path is provided
    # if model_path and os.path.exists(model_path):
    #     print(f"Loading pre-trained model from {model_path}")
    #     model.load_state_dict(torch.load(model_path, map_location=device))
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

    print("Opening Label.json!")
    with open('/kaggle/input/train-data-new/labels.json', 'r') as file:
        label_map = json.load(file)

    model.train()

    start_time = time.time()
    for epoch in range(num_epochs):
        train_loader = get_data_loaders(frame_direc, batch_size=batch_size)
        
        batch_count = 0
        for data, video_names in train_loader:
            if data is None:
                print("No valid data returned from loader, skipping this batch.")
                continue

            data = data.to(device)

            optimizer.zero_grad()
            outputs = model(data)
            outputs = outputs.squeeze(1)

            labels_tensor = []
            for video_name in video_names:
                label = label_map.get(video_name)
                # Replace -1 with 0
                if label == -1:
                    label = 0
                labels_tensor.append(label)

            labels_tensor = torch.tensor(labels_tensor).float().to(device)

            loss = criterion(outputs, labels_tensor)
            loss.backward()
            optimizer.step()

            if (batch_count + 1) % 20 == 0:
                epoch_time = time.time() - start_time
                # epoch_time = end_time - 
                start_time = time.time()
                print(f'Batch {batch_count+1}/{len(train_loader)} // {epoch+1}/{num_epochs}, Loss: {loss.item()}, Time: {epoch_time}')
            batch_count += 1
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
        
        # Save checkpoint every 5 epochs
        if (epoch + 1) % 5 == 0:
            save_path = f'/kaggle/working/models/{model_name}_model_epoch_{epoch+1}.pth'
            os.makedirs(os.path.dirname(save_path), exist_ok=True)  # Make sure the folder exists
            torch.save(model.state_dict(), save_path)
            print(f'Checkpoint saved at epoch {epoch+1} to {save_path}')
        
    # Save the model for the last epoch as well
    save_path = f'/kaggle/working/models/{model_name}_model_epoch_{num_epochs}.pth'
    os.makedirs(os.path.dirname(save_path), exist_ok=True)  # Make sure the folder exists
    torch.save(model.state_dict(), save_path)
    print(f'Final model saved at epoch {num_epochs} to {save_path}')


def main():
    num_epochs = 25
    # frame_direc = 'testing_output'
    frame_direc = '/kaggle/input/train-output-24/train_output_24'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("Working with device: ", device)
    
    # Add command line argument for model path
    # import argparse
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--model_path', type=str, default=None, help='Path to pre-trained model')
    # parser.add_argument('--model_name', type=str, default='two_stream', help='Name of trained model')
    # args = parser.parse_args()
    model_name = 'two_stream_24'
    
    train_model(num_epochs, frame_direc, device, 24, model_name)

if __name__ == "__main__":
    main()

import torch

torch.cuda.empty_cache()

# Check if CUDA is available
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory Allocated: {torch.cuda.memory_allocated(i) / 1024 ** 2} MB")
        print(f"  Memory Cached: {torch.cuda.memory_reserved(i) / 1024 ** 2} MB")
else:
    print("CUDA is not available on this machine.")


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
from models_attention import TwoStreamNetworkTransferLearning
from data_loader_new_train import get_data_loaders
import json
import os
import time

def get_device_for_model():
    # Check memory usage on all available GPUs
    num_gpus = torch.cuda.device_count()
    for i in range(num_gpus):
        mem_alloc = torch.cuda.memory_allocated(i)
        mem_total = torch.cuda.get_device_properties(i).total_memory
        if mem_alloc < mem_total * 0.95:
            return torch.device(f'cuda:{i}')
    raise MemoryError("All GPUs are out of memory!")

def train_model(num_epochs, frame_direc, device, batch_size=1, model_name=None, start_epoch=0):
    print("Entering to train data!")
    model = TwoStreamNetworkTransferLearning()

    # Use DataParallel with all available GPUs
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))
    
    device = get_device_for_model()
    model = model.to(device)
    
    # Load pre-trained model if path is provided
    checkpoint_path = f'/kaggle/working/models/{model_name}_model_epoch_{start_epoch}.pth'
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
    
    if os.path.exists(checkpoint_path):
        print(f"Loading model checkpoint from {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        
        # Handle DataParallel state dict
        state_dict = checkpoint['model_state_dict']
        if isinstance(model, nn.DataParallel):
            model.module.load_state_dict(state_dict)
        else:
            model.load_state_dict(state_dict)
            
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"Resuming training from epoch {start_epoch}")
    
    criterion = nn.BCELoss()

    print("Opening Label.json!")
    with open('/kaggle/input/train-data-new/labels.json', 'r') as file:
        label_map = json.load(file)

    model.train()

    # Add gradient accumulation for larger effective batch size
    accumulation_steps = 4
    start_time = time.time()
    
    for epoch in range(start_epoch, num_epochs):
        train_loader = get_data_loaders(frame_direc, batch_size=batch_size)
        
        batch_count = 0
        optimizer.zero_grad()
        
        for i, (data, video_names) in enumerate(train_loader):
            if data is None:
                print("No valid data returned from loader, skipping this batch.")
                continue

            data = data.to(device)
            outputs = model(data).squeeze(1)
            
            labels_tensor = torch.tensor([
                0 if label_map.get(video_name) == -1 else label_map.get(video_name)
                for video_name in video_names
            ]).float().to(device)

            loss = criterion(outputs, labels_tensor)
            loss = loss / accumulation_steps  # Normalize loss
            loss.backward()

            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            if (batch_count + 1) % 20 == 0:
                epoch_time = time.time() - start_time
                start_time = time.time()
                print(f'Batch {batch_count+1}/{len(train_loader)} // {epoch+1}/{num_epochs}, Loss: {loss.item()}, Time: {epoch_time}')
            batch_count += 1
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
        
        # Save checkpoint every epoch with proper DataParallel handling
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss.item(),
        }
        save_path = f'/kaggle/working/models/{model_name}_model_epoch_{epoch+1}.pth'
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        torch.save(checkpoint, save_path)
        print(f'Checkpoint saved at epoch {epoch+1} to {save_path}')

def clear_gpu_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_max_memory_allocated()
        torch.cuda.reset_max_memory_cached()
        print("GPU cache cleared")
    else:
        print("No GPU available")

def main():
    # clear_gpu_cache()
    num_epochs = 25
    frame_direc = '/kaggle/input/train-output-24-reduced/train_output_24_reduced'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("Working with device: ", device)
    
    model_name = 'two_stream_24'
    start_epoch = 0  # Set this to the epoch you want to continue from
    
    train_model(num_epochs, frame_direc, device, 20, model_name, start_epoch)

if __name__ == "__main__":
    main()

In [13]:
# COPY FILES TO WORKING DIRECTORY

import os
from shutil import copyfile

# Define the source and destination directorie
src_dir = '/kaggle/input/train-data-new/'  # Source directory
dst_dir = '/kaggle/working/'  # Destination directory

# Ensure the destination directory exists
os.makedirs(dst_dir, exist_ok=True)

# Iterate over all the files in the source directory
for filename in os.listdir(src_dir):
    # Get the full file path
    src_file = os.path.join(src_dir, filename)
    
    # Ensure that it's a file (not a directory)
    if os.path.isfile(src_file):
        # Define the destination file path
        dst_file = os.path.join(dst_dir, filename)
        
        # Copy the file
        copyfile(src_file, dst_file)
        print(f'Copied {filename} to {dst_file}')


Copied real_labels_not_used.json to /kaggle/working/real_labels_not_used.json
Copied models_attention.py to /kaggle/working/models_attention.py
Copied labels.json to /kaggle/working/labels.json
Copied test.py to /kaggle/working/test.py
Copied train_attention.py to /kaggle/working/train_attention.py
Copied vaf_util_new.py to /kaggle/working/vaf_util_new.py
Copied data_loader_new_train.py to /kaggle/working/data_loader_new_train.py
Copied real_labels.json to /kaggle/working/real_labels.json
Copied vaf_ext.py to /kaggle/working/vaf_ext.py
Copied data_loader_test.py to /kaggle/working/data_loader_test.py


In [ ]:
import tensorflow as tf

# Clear the current TensorFlow session
tf.keras.backend.clear_session()

# Check memory usage with nvidia-smi
import subprocess
subprocess.run(['nvidia-smi'])

In [15]:
def clear_gpu_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_max_memory_allocated()
        torch.cuda.reset_max_memory_cached()
        print("GPU cache cleared")
    else:
        print("No GPU available")

clear_gpu_cache()

GPU cache cleared


In [14]:
import os

def kill_gpu_processes():
    # Get list of processes using GPU
    processes = os.popen('nvidia-smi --query-compute-apps=pid --format=csv,noheader').read().strip().split('\n')
    
    # Kill each process
    for pid in processes:
        if pid:  # Check if pid is not empty
            os.system(f'kill -9 {pid}')
    
    print(f"Killed {len(processes)} GPU processes")

# Usage
kill_gpu_processes()

Killed 2 GPU processes


In [25]:
import torch
import gc

def reset_all_gpus():
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            # Set the GPU device
            torch.cuda.set_device(i)
            
            # Clear memory for the current GPU
            gc.collect()
            torch.cuda.empty_cache()
            torch.cuda.reset_max_memory_allocated()
            torch.cuda.reset_max_memory_cached()
            torch.cuda.ipc_collect()
            print(f"GPU {i} fully reset")
    else:
        print("No GPU available")

reset_all_gpus()


GPU 0 fully reset
GPU 1 fully reset


In [ ]:
## TRAIN MODEL

import torch
import torch.optim as optim
import torch.nn as nn
from models_attention import TwoStreamNetworkTransferLearning
from data_loader_new_train import get_data_loaders
# from vaf_ext.py import 
import json
import os
import time

def get_device_for_model():
    # Check memory usage on all available GPUs
    num_gpus = torch.cuda.device_count()
    for i in range(num_gpus):
        mem_alloc = torch.cuda.memory_allocated(i)
        mem_total = torch.cuda.get_device_properties(i).total_memory
        if mem_alloc < mem_total * 0.98:
            return torch.device(f'cuda:{i}')
    raise MemoryError("All GPUs are out of memory!")

def train_model(num_epochs, frame_direc, device, batch_size=1, model_name=None, start_epoch=0):
    print("Entering to train data!")
    model = TwoStreamNetworkTransferLearning()

    # Use DataParallel with all available GPUs
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))
    
    device = get_device_for_model()
    model = model.to(device)
    
    # Load pre-trained model if path is provided
    checkpoint_path = f'models/{model_name}_model_epoch_{start_epoch}.pth'
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
    
    if os.path.exists(checkpoint_path):
        print(f"Loading model checkpoint from {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        
        # Handle DataParallel state dict
        state_dict = checkpoint['model_state_dict']
        if isinstance(model, nn.DataParallel):
            model.module.load_state_dict(state_dict)
        else:
            model.load_state_dict(state_dict)
            
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"Resuming training from epoch {start_epoch}")
    
    criterion = nn.BCELoss()

    print("Opening Label.json!")
    with open('/kaggle/input/train-data-new/labels.json', 'r') as file:
    # with open('labels.json', 'r') as file:
        label_map = json.load(file)

    model.train()

    # Add gradient accumulation for larger effective batch size
    accumulation_steps = 4
    start_time = time.time()
    
    for epoch in range(start_epoch, num_epochs):
        train_loader = get_data_loaders(frame_direc, batch_size=batch_size)
        
        batch_count = 0
        optimizer.zero_grad()
        
        for i, (data, video_names, vaf_features) in enumerate(train_loader):
            if data is None:
                print("No valid data returned from loader, skipping this batch.")
                continue

            # print(video_names)
            # print(type(data), data.shape)
            # print(type(vaf_features), len(vaf_features), len(vaf_features[0]))

            data = data[:, 12*2:, :, :, :]
            # vaf_features = vaf_features.view(-1, 24, 4)
            # print("data shape:", data.shape)
            # print("vaf_features shape:", vaf_features.shape)
            data = data.float().to(device)
            vaf_features = vaf_features.float().to(device) # Force [batch_size, 96]



            # # Ensure VAF features are [batch_size, 24, 4]
            # if vaf_features.dim() == 2:  # If flattened
            #     vaf_features = vaf_features.view(-1, 24, 4)

            # print("data shape:", data.shape)
            # print("vaf_features shape:", vaf_features.shape)

            outputs = model(data, vaf_features).squeeze(1)
            # outputs = model(data, vaf_features).squeeze(1)
            
            # print(video_names)

            labels_tensor = torch.tensor([
                0 if label_map.get(video_name) == -1 else label_map.get(video_name)
                for video_name in video_names
            ]).float().to(device)

            loss = criterion(outputs, labels_tensor)
            loss = loss / accumulation_steps  # Normalize loss
            loss.backward()

            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            if (batch_count + 1) % 20 == 0:
                epoch_time = time.time() - start_time
                start_time = time.time()
                print(f'Batch {batch_count+1}/{len(train_loader)} // {epoch+1}/{num_epochs}, Loss: {loss.item()}, Time: {epoch_time}')
            batch_count += 1
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
        
        # Save checkpoint every epoch with proper DataParallel handling
        if (epoch + 1) % 5 == 0 or epoch == 0 or (epoch + 1) == num_epochs:
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss.item(),
            }
            
            save_path = f'/kaggle/working/models/{model_name}_model_epoch_{epoch+1}.pth'
            # save_path = f'/models/{model_name}_model_epoch_{epoch+1}.pth'
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            torch.save(checkpoint, save_path)
            print(f'Checkpoint saved at epoch {epoch+1} to {save_path}')


# import torch

def reset_all_gpus():
    import gc
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            # Set the GPU device
            torch.cuda.set_device(i)
            
            # Clear memory for the current GPU
            gc.collect()
            torch.cuda.empty_cache()
            torch.cuda.reset_max_memory_allocated()
            torch.cuda.reset_max_memory_cached()
            torch.cuda.ipc_collect()
            print(f"GPU {i} fully reset")
    else:
        print("No GPU available")



# Usage

def main():
    num_epochs = 15
    frame_direc = '/kaggle/input/train-output-24-reduced/train_output_24_reduced'
    # frame_direc = '/media/edward/OS/Users/arind/test_output_24/'
    # frame_direc = './testing_sample_data'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # device = 'cpu'
    print("Working with device: ", device)
    
    model_name = 'two_stream_24'
    start_epoch = 0  # Set this to the epoch you want to continue from
    
    reset_all_gpus()
    # clear_gpu_cache()
    train_model(num_epochs, frame_direc, device, 48, model_name, start_epoch)

if __name__ == "__main__":
    main()

Working with device:  cuda
GPU 0 fully reset
GPU 1 fully reset
Entering to train data!
Using 2 GPUs!
Opening Label.json!


In [9]:
## TEST MODEL

import torch
import torch.nn as nn
from models_attention import TwoStreamNetworkTransferLearning
# from data_loader_new_train import get_data_loaders
from data_loader_test import get_test_loaders
import json
import os
import argparse

def get_device_for_model():
    num_gpus = torch.cuda.device_count()
    for i in range(num_gpus):
        mem_alloc = torch.cuda.memory_allocated(i)
        mem_total = torch.cuda.get_device_properties(i).total_memory
        if mem_alloc < mem_total * 0.95:
            return torch.device(f'cuda:{i}')
    raise MemoryError("All GPUs are out of memory!")

def test_model(frame_direc, device, batch_size, threshold=0.5):
    model = TwoStreamNetworkTransferLearning()
    
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(torch.cuda.device_count())))
    
    device = get_device_for_model()
    model = model.to(device)
    
    checkpoint_path = '/kaggle/input/two_stream_24_model_epoch_15/pytorch/default/1/two_stream_24_model_epoch_15.pth'
    if os.path.exists(checkpoint_path):
        print(f"Loading model checkpoint from {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        
        state_dict = checkpoint['model_state_dict']
        if isinstance(model, nn.DataParallel):
            model.module.load_state_dict(state_dict)
        else:
            model.load_state_dict(state_dict)
    
    model.eval()

    print("Opening Label.json!")
    # with open('real_labels.json', 'r') as file:
    with open('real_labels_not_used.json', 'r') as file:
        label_map = json.load(file)

    correct = 0
    total = 0

    test_loader = get_test_loaders(frame_direc, batch_size)
    # test_loader = get_data_loaders(frame_direc, batch_size)
    with torch.no_grad():
        for data, video_names, vaf_features in test_loader:
            if data is None:
                continue
                
            print(f"Processing batch: {video_names}")
            data = data[:, 12*2:, :, :, :].float().to(device)
            vaf_features = vaf_features.float().to(device)
            
            outputs = model(data, vaf_features).squeeze(1)
            
            labels_tensor = torch.tensor([
                0 if label_map.get(video_name) == -1 else label_map.get(video_name)
                for video_name in video_names
            ]).float().to(device)
            
            predicted = (outputs >= threshold).float()
            total += labels_tensor.size(0)
            correct += (predicted == labels_tensor).sum().item()

            print(f"Model outputs: {outputs.cpu().numpy()}")
            print(f"Predictions: {predicted.cpu().numpy()}")
            print(f"Ground truth: {labels_tensor.cpu().numpy()}")

    accuracy = correct / total * 100
    print(f'Final Accuracy: {accuracy:.2f}% (Threshold: {threshold})')

def reset_gpu():
    import gc
    if torch.cuda.is_available():
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.reset_max_memory_allocated()
        torch.cuda.reset_max_memory_cached()
        torch.cuda.ipc_collect()
        print("GPU fully reset")
    else:
        print("No GPU available")

if __name__ == "__main__":
    reset_gpu()
    parser = argparse.ArgumentParser()
    # parser.add_argument('--threshold', type=float, default=0.75, help='Threshold for binary classification')
    # args = parser.parse_args()

    # frame_direc = '/media/edward/OS/Users/arind/test_output_24'
    # frame_direc = '/media/edward/OS/Users/arind/train_output_24/not_training_on_this'
    frame_direc = '/kaggle/input/test-data/not_training_on_this'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    test_model(frame_direc, device, 18, 0.99)

GPU fully reset
Using 2 GPUs!
Loading model checkpoint from /kaggle/input/two_stream_24_model_epoch_15/pytorch/default/1/two_stream_24_model_epoch_15.pth


<ipython-input-9-ffa60bdd6319>:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


Opening Label.json!
Processing batch: ('id35_id17_0002.mp4', 'id33_id30_0001.mp4', 'id8_id7_0003.mp4', 'id43_id45_0006.mp4', 'id26_id0_0000.mp4', 'id3_0002.mp4', 'id9_id26_0007.mp4', 'id31_id26_0007.mp4', 'id24_id20_0008.mp4', 'id9_id0_0000.mp4', 'id41_id44_0001.mp4', 'id28_id27_0004.mp4', 'id47_id43_0007.mp4', 'id23_id38_0004.mp4', 'id35_id9_0006.mp4', 'id16_id1_0009.mp4', 'id2_id1_0001.mp4', 'id26_id37_0003.mp4')
Model outputs: [0.52515304 0.2500933  0.18029445 0.47880247 0.9990114  0.9212203
 0.9913059  0.9737665  0.52910167 0.99732983 0.4592768  0.7084442
 0.95174724 0.8908359  0.9973713  0.25597185 0.96530783 0.9961688 ]
Predictions: [0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1.]
Ground truth: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Processing batch: ('id23_id32_0002.mp4', 'id43_id48_0006.mp4', 'id10_id11_0004.mp4', 'id35_id31_0006.mp4', 'id38_id28_0008.mp4', 'id54_id53_0001.mp4', 'id6_id16_0005.mp4', 'id1_0004.mp4', 'id20_id37_0005.mp4', 'id16_id21_0004.m